In [12]:
import mediapipe as mp
import cv2
from dollarpy import Recognizer, Template, Point
import time
import pickle
import socket
import threading

mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic


In [13]:
# Load the model
model_filename = 'thumbsup.pkl'
with open(model_filename, 'rb') as model_file:
    templates = pickle.load(model_file)

# Initialize recognizer with loaded templates
recognizer = Recognizer(templates)
print("Model loaded successfully.")

Model loaded successfully.


In [32]:
import socket
import cv2
import mediapipe as mp
import time

# Set up MediaPipe components for gesture detection
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
thumbs_up_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
thumbs_up_socket.connect(('DESKTOP-8161GCK', 8001))  # Connect to the main script
print("Connected to the server.")
def thumbs_up_status_callback(thumbs_up_detected):

    if thumbs_up_detected:
        print("Thumbs up detected, attempting to send signal")
        try:
            # Initialize client socket for thumbs-up detection
            thumbs_up_socket.sendall(b'thumbs_up')  # Send thumbs-up signal
            print("Thumbs-up signal sent successfully")
        except Exception as e:
            print("Error sending thumbs-up signal:", e)

def getPointsRealTime(thumbs_up_detected_callback):
    cap = cv2.VideoCapture(0)  # Use camera
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Flip the frame horizontally to mirror the camera feed
            frame = cv2.flip(frame, 1)

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            results = holistic.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Draw hand landmarks
            mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

            # Check for thumbs-up gesture
            thumbs_up_detected = False
            if results.right_hand_landmarks or results.left_hand_landmarks:
                try:
                    hand_landmarks = results.right_hand_landmarks or results.left_hand_landmarks

                    # Get the coordinates for landmarks needed for thumbs-up detection
                    thumb_tip = hand_landmarks.landmark[mp.solutions.hands.HandLandmark.THUMB_TIP]
                    thumb_ip = hand_landmarks.landmark[mp.solutions.hands.HandLandmark.THUMB_IP]
                    index_mcp = hand_landmarks.landmark[mp.solutions.hands.HandLandmark.INDEX_FINGER_MCP]
                    middle_mcp = hand_landmarks.landmark[mp.solutions.hands.HandLandmark.MIDDLE_FINGER_MCP]

                    # Condition to check if thumb is up and other fingers are folded
                    if (thumb_tip.y < index_mcp.y and thumb_tip.y < middle_mcp.y) and \
                       (thumb_tip.x > thumb_ip.x):  # Thumb pointing upwards condition

                        thumbs_up_detected = True
                        thumb_x = int(thumb_tip.x * image.shape[1])
                        thumb_y = int(thumb_tip.y * image.shape[0])
                        cv2.circle(image, (thumb_x, thumb_y), 10, (0, 255, 0), -1)  # Draw green dot

                except Exception as e:
                    print(f"Error processing hand landmarks: {e}")

            # Call the callback with the current thumbs-up status
            thumbs_up_detected_callback(thumbs_up_detected)

            # Display the frame
            cv2.imshow("Gesture Detection", image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
                thumbs_up_socket.close()  # Close the connection after sending


        cap.release()
        cv2.destroyAllWindows()

# Start real-time gesture detection
getPointsRealTime(thumbs_up_status_callback)


Connected to the server.
Thumbs up detected, attempting to send signal
Thumbs-up signal sent successfully
Thumbs up detected, attempting to send signal
Thumbs-up signal sent successfully
Thumbs up detected, attempting to send signal
Error sending thumbs-up signal: [WinError 10053] An established connection was aborted by the software in your host machine
Thumbs up detected, attempting to send signal
Error sending thumbs-up signal: [WinError 10053] An established connection was aborted by the software in your host machine
